# Add extra information to DRZ auction results

Query to the open data dataset of the RDW.

It may take a while (10 min) to query all auction results.
- - - - 

### User variables


In [1]:
Date = '2019-04' # yyyy-mm

### Modules and functions

In [2]:
import pandas as pd
import re 
# to keep api key hidden import this from sub dir
import assets.hidden_api_keys as hidden_api_keys

# base url
apiurl = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json?$$app_token=' + hidden_api_keys.socrata_apptoken + '&'

def get_json_from_api(url,reg,c=0):
    
    '''Get json object from api'''
    
    import time

    c+=1
    try:
        df=pd.read_json(url + 'kenteken=' + reg.replace('-','').upper()).to_dict()
    except:
        if c > 10:
            print(url,reg)
            raise 
        else:
            print('pause 2 sec and try again!')
            time.sleep(2)
            df = get_json_from_api(url,reg,c)
    
    return df
    
# get_json_from_api(apiurl,'61-sf-FG')

### Load auction results

In [3]:
file_name = '../data/drz-data-{}.pkl'.format(Date)
print(file_name)
drz = pd.read_pickle(file_name)

../data/drz-data-2019-04.pkl


### query rdw

In [4]:
# see what lots have a Dutch registration (license number).
hasReg = (~drz.Reg.isnull()) & (drz.Reg != 'onbekend') & (drz.Reg != '') & (~drz.LotType.isin([
    'Vaartuig',
    'Jetski',
    'Sloep',
    'Speedboot',
    'Vaartuig (Type onbekend)',
    'Motorvaartuig met opbouw (Pleziervaartuig)',
    'Aanhangwagen',
]))

# make a copy and add info
rdw = drz.copy()


In [5]:
def get_query_url(api_url, regs, token=hidden_api_keys.socrata_apptoken):
    '''construct query url'''
    
    # convert list to string
    reg_list = ''.join(["'{}', ".format(r.replace('-','').upper()) for r in regs])
    reg_list = reg_list[0:-2] # remove trailing ', '
    q = api_url + '?$$app_token=' + token + '&$where='
    # add escaped soql
    q += urllib.parse.quote('kenteken in(' + reg_list + ')')
    return q

Create list of dataframes with different api results

In [6]:
import urllib

# empty list
dfs = list()
# first element is all registrations
dfs.append(rdw.loc[hasReg,['Reg']])
dfs[0]['kenteken'] = dfs[0].Reg.apply(lambda r: r.replace('-','').upper())
dfs[0].index.name = 'lot_index'
dfs[0] = dfs[0].reset_index().set_index('kenteken')
display(dfs[0].tail())
print(dfs[0].shape)


# do main api first to get other possible apis
api_name = 'api_gekentekende_voertuigen'
api_url = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json'
regs = dfs[0].Reg.values
print(len(regs),'registrations in this set')

# query data base
q = get_query_url(api_url,regs)
dfs.append(pd.read_json(q))
dfs[1].set_index('kenteken', inplace=True)
display(dfs[1].tail())
print(dfs[1].shape)

# query other available apis
for api_name in [c for c in dfs[1].columns if c.startswith('api')]:
    print(api_name)
    for api_url in dfs[1][api_name].unique():
        print(api_url)
        # query the web
        q = get_query_url(api_url,regs)
        df0 = pd.read_json(q)
        # name of index
        df0.columns.name = api_name

        # query should return 'kenteken', make it the index
        if df0.shape[0] != 0:
            df0.set_index('kenteken', inplace=True)
            
        # Some apis return multiple values. Pivot around index number ("volgnummer")
        if api_name == 'api_gekentekende_voertuigen_assen':
            df0 = pd.pivot(df0, columns='as_nummer')

        elif api_name == 'api_gekentekende_voertuigen_brandstof':
            df0 = pd.pivot(df0, columns='brandstof_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie_specifiek':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        # squeeze multi index
        one_level = [
            re.sub('^api_gekentekende_voertuigen_','',api_name) + '_' + '_'.join(
                [str(c) if type(c)==int else c for c in l]
            ) for l in df0.columns
        ]
        df0.columns = one_level

        # add to list
        display(df0.tail())
        print(df0.shape)
        dfs.append(df0)   

,lot_index,Reg
kenteken,,
20WFZR,2019-4-7343,20-WF-ZR
86WSKS,2019-4-7344,86-WS-KS
BVXS15,2019-4-7348,BV-XS-15
BNTP31,2019-4-7349,BN-TP-31
BRJR42,2019-4-7351,BR-JR-42


(214, 2)
214 registrations in this set


,aanhangwagen_autonoom_geremd,aanhangwagen_middenas_geremd,aantal_cilinders,aantal_deuren,aantal_rolstoelplaatsen,aantal_wielen,aantal_zitplaatsen,afstand_hart_koppeling_tot_achterzijde_voertuig,afstand_voorzijde_voertuig_tot_hart_koppeling,api_gekentekende_voertuigen_assen,...,vermogen_brom_snorfiets,vermogen_massarijklaar,vervaldatum_apk,vervaldatum_tachograaf,voertuigsoort,volgnummer_wijziging_eu_typegoedkeuring,wacht_op_keuren,wam_verzekerd,wielbasis,zuinigheidslabel
kenteken,,,,,,,,,,,,,,,,,,,,,
VRFB58,NaN,NaN,4.0,0.0,NaN,4.0,NaN,0.0,0.0,https://opendata.rdw.nl/resource/3huj-srit.json,...,NaN,0.03,20180511.0,NaN,Bedrijfsauto,0.0,Geen verstrekking in Open Data,Nee,292,NaN
VZ055J,NaN,2500.0,4.0,0.0,0.0,4.0,3.0,0.0,0.0,https://opendata.rdw.nl/resource/3huj-srit.json,...,NaN,0.07,20190908.0,NaN,Bedrijfsauto,0.0,Geen verstrekking in Open Data,Ja,328,NaN
VZ744H,2000.0,2000.0,4.0,0.0,0.0,4.0,3.0,0.0,0.0,https://opendata.rdw.nl/resource/3huj-srit.json,...,NaN,0.04,20190901.0,NaN,Bedrijfsauto,0.0,Geen verstrekking in Open Data,Nee,367,NaN
XS82HB,NaN,NaN,4.0,3.0,NaN,4.0,NaN,0.0,0.0,https://opendata.rdw.nl/resource/3huj-srit.json,...,NaN,0.02,20170705.0,NaN,Personenauto,0.0,Geen verstrekking in Open Data,Nee,365,NaN
XXZT91,NaN,NaN,6.0,4.0,0.0,4.0,5.0,0.0,0.0,https://opendata.rdw.nl/resource/3huj-srit.json,...,NaN,0.08,20190824.0,NaN,Personenauto,1.0,Geen verstrekking in Open Data,Ja,275,NaN


(208, 60)
api_gekentekende_voertuigen_assen
https://opendata.rdw.nl/resource/3huj-srit.json


,assen_aangedreven_as_1,assen_aangedreven_as_2,assen_aantal_assen_1,assen_aantal_assen_2,assen_hefas_1,assen_hefas_2,assen_plaatscode_as_1,assen_plaatscode_as_2,assen_spoorbreedte_1,assen_spoorbreedte_2,assen_technisch_toegestane_maximum_aslast_1,assen_technisch_toegestane_maximum_aslast_2,assen_weggedrag_code_1,assen_weggedrag_code_2,assen_wettelijk_toegestane_maximum_aslast_1,assen_wettelijk_toegestane_maximum_aslast_2
kenteken,,,,,,,,,,,,,,,,
VRFB58,NaN,NaN,2,2,NaN,NaN,V,A,161.0,157.0,NaN,NaN,NaN,NaN,1250,1230
VZ055J,J,N,2,2,NaN,NaN,V,A,163.0,162.0,1500.0,1800.0,NaN,NaN,1500,1800
VZ744H,N,J,2,2,NaN,NaN,V,A,171.0,172.0,1650.0,2250.0,NaN,NaN,1650,2250
XS82HB,NaN,NaN,2,2,NaN,NaN,V,A,168.0,165.0,NaN,NaN,NaN,NaN,1550,1680
XXZT91,NaN,NaN,2,2,NaN,NaN,NaN,NaN,158.0,159.0,1145.0,1070.0,NaN,NaN,1145,1070


...

(171, 2)
api_gekentekende_voertuigen_carrosserie_specifiek
https://opendata.rdw.nl/resource/jhie-znh9.json


,carrosserie_specifiek_carrosserie_voertuig_nummer_code_volgnummer_1,carrosserie_specifiek_carrosserie_voertuig_nummer_europese_omschrijving_1,carrosserie_specifiek_carrosseriecode_1
kenteken,,,
7VTB43,1,Gesloten opbouw,3
86WSKS,1,Gesloten opbouw,3
BNHR29,1,Voor vervoer voertuigen,14
V202GT,1,Gesloten opbouw,3
V718FB,1,Gesloten opbouw,3


(8, 3)
api_gekentekende_voertuigen_voertuigklasse
https://opendata.rdw.nl/resource/kmfi-hrps.json


""


(0, 0)


In [7]:
# Merge dataframes from different apis
df = pd.concat(dfs, axis='columns', sort=False)
# add timestamp
df['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')
# set lot id as index
df.index.name = 'kenteken'
df = df.reset_index().set_index('lot_index')
display(df.tail())
print(df.shape)

,kenteken,Reg,aanhangwagen_autonoom_geremd,aanhangwagen_middenas_geremd,aantal_cilinders,aantal_deuren,aantal_rolstoelplaatsen,aantal_wielen,aantal_zitplaatsen,afstand_hart_koppeling_tot_achterzijde_voertuig,...,brandstof_uitstoot_deeltjes_licht_1,brandstof_uitstoot_deeltjes_licht_2,brandstof_uitstoot_deeltjes_zwaar_1,brandstof_uitstoot_deeltjes_zwaar_2,carrosserie_carrosserietype_1,carrosserie_type_carrosserie_europese_omschrijving_1,carrosserie_specifiek_carrosserie_voertuig_nummer_code_volgnummer_1,carrosserie_specifiek_carrosserie_voertuig_nummer_europese_omschrijving_1,carrosserie_specifiek_carrosseriecode_1,TimeStamp
lot_index,,,,,,,,,,,,,,,,,,,,,
2019-4-7343,20WFZR,20-WF-ZR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380.0,...,NaN,NaN,NaN,NaN,DC,Middenasaanhangwagen,1.0,Veewagen,13.0,20190412
2019-4-7344,86WSKS,86-WS-KS,NaN,NaN,NaN,0.0,0.0,4.0,NaN,538.0,...,NaN,NaN,NaN,NaN,DC,Middenasaanhangwagen,1.0,Gesloten opbouw,3.0,20190412
2019-4-7348,BVXS15,BV-XS-15,NaN,NaN,6.0,NaN,NaN,4.0,2.0,NaN,...,NaN,NaN,NaN,NaN,BC,Opleggertrekker,NaN,NaN,NaN,20190412
2019-4-7349,BNTP31,BN-TP-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190412
2019-4-7351,BRJR42,BR-JR-42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190412


(214, 114)


### Saving

In [8]:
file_name = '../data/rdw-data-{}.pkl'.format(Date)
print(file_name)
rdw.to_pickle(file_name)

../data/rdw-data-2019-04.pkl
